## Fine-Tuning GPT2 with pytorch




*   Import all necessary libraries



In [ ]:
import os
import time
import datetime
from google.colab import drive

import pandas as pd
import seaborn as sns
import numpy as np
import random

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install transformers


*   Download 20 news groups using the sklearn library in Python
*   Acess the text data and store them in data.

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# Download the dataset
newsgroups_data = fetch_20newsgroups(subset='all')

# Access the text data and labels
data = newsgroups_data.data

In [ ]:
data[0]

"From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>\nSubject: Pens fans reactions\nOrganization: Post Office, Carnegie Mellon, Pittsburgh, PA\nLines: 12\nNNTP-Posting-Host: po4.andrew.cmu.edu\n\n\n\nI am sure some bashers of Pens fans are pretty confused about the lack\nof any kind of posts about the recent Pens massacre of the Devils. Actually,\nI am  bit puzzled too and a bit relieved. However, I am going to put an end\nto non-PIttsburghers' relief with a bit of praise for the Pens. Man, they\nare killing those Devils worse than I thought. Jagr just showed you why\nhe is much better than his regular season stats. He is also a lot\nfo fun to watch in the playoffs. Bowman should let JAgr have a lot of\nfun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final\nregular season game.          PENS RULE!!!\n\n"



* Preprocessing the dataset "data" to extract the subject  and the message body from each message.
* Formating it to a specific structure with the subject followed by a summary indicator ('; TLDR: ') and the message body.





In [ ]:
import re

newsgroup_messages = []

for el in data:
  lines = el.split('\n')

  body, subject, element = '', '', ''

  for line in lines:
    # save the subject
    if 'Subject:' in line:
      subject = line[len('Subject:') + 1:]

    # ignoring other headers
    elif len(re.findall("^[A-Za-z-_\.]+:", line)) != 0:
      continue

    # save the body
    else:
      body += line
  # input: subject ; TLDR: body
  # we put backwards so that the model learns to generate the continuation
  element = subject + '; TLDR: ' + body
  newsgroup_messages.append(element)

data = newsgroup_messages

In [ ]:
data[0]

"Pens fans reactions; TLDR: I am sure some bashers of Pens fans are pretty confused about the lackof any kind of posts about the recent Pens massacre of the Devils. Actually,I am  bit puzzled too and a bit relieved. However, I am going to put an endto non-PIttsburghers' relief with a bit of praise for the Pens. Man, theyare killing those Devils worse than I thought. Jagr just showed you whyhe is much better than his regular season stats. He is also a lotfo fun to watch in the playoffs. Bowman should let JAgr have a lot offun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the finalregular season game.          PENS RULE!!!"



*   Setting GPT2 Tokenizer



In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]



*   Defining a custom dataset 'GPT2Dataset' for pytorch which will be used for model.



In [ ]:
batch_size = 2

class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]





*   Creating the custom dataset
*   Then, dividing this dataset into two parts: Train qnd validation




In [ ]:
dataset = GPT2Dataset(data, tokenizer, max_length=768)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

16,961 training samples
1,885 validation samples




*   Setting up data loaders for training and validation using PyTorch's
 to efficiently load and process batches of data during training and validation.

In [ ]:
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )



*    Loading the GPT-2 configuration with pre-trained weights and disables the output of hidden states.
*   Initiating GPT2 model using the GPT2LMHeadModel class.
*   Resizing the token embeddings of the model to match the vocabulary size of the GPT2 tokenizer (This step is necessary since we added special tokens (bos_token..) to the embeddings, ensuring alignment between the tokenizer and the model)





In [ ]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda")
model.cuda()

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

# Note: AdamW is a class from the huggingface library (as opposed to pytorch)
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()

        outputs = model(  b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            # "Nucleus Sampling" strategy, which is also known as "Top-p Sampling" or "Top-k Sampling."
            sample_outputs = model.generate(
                                    bos_token_id=random.randint(1,30000),
                                    do_sample=True,
                                    top_k=50,
                                    max_length = 200,
                                    top_p=0.95,
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(b_input_ids,
#                            token_type_ids=None,
                             attention_mask = b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 5 ========
Training...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of  8,481. Loss: 2.3331193923950195.   Elapsed: 0:00:47.
0:  bipartisan" David Samuel Johnson: Why don't we "mildly agree" with the "mild" version of the Senate bill? Democrat senator: With good reason. There are Republicans who say the Senate bill is not "too much like it's bad" Democrat                                                                                                                                             


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   200  of  8,481. Loss: 1.6101561784744263.   Elapsed: 0:01:38.
0:  increasingSophia, the word appears more often in the                                                                                                                                                                                           


...

  Batch 8,200  of  8,481. Loss: 0.3793332874774933.   Elapsed: 1:13:32.
0:  CreativeRe: "Hardware that Fits" mail order; TLDR: kdb@sunbar.mc.duke.edu (Kurt Bollacker) writes:>In article <1993Apr22.221344.15950@magnus.acs.ohio-state.edu> jnielsen@magnus.acs.ohio-state.edu (John F Nielsen) writes:>>>...  Just in case it's important, I've got a couple of questions to ask you.>>>> I mail a set of mail order address's out of the blue box:>>  1-800-832-5200.  2- If you're interested, I can ship you a set off the ground>>    in Seattle, and you can't use them in your mail.  If you're not, then you can't use them>    anywhere.  If you're not, then it's the other way around


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 8,300  of  8,481. Loss: 0.299060195684433.   Elapsed: 1:14:27.
0: ImageRe: HELP for Kidney Stones..............; TLDR: In article <7166@blue.cis.pitt.edu>, jhaines@pitt.edu (Josh He) says:>In article <C697IJ.663@wetware.com> drieux@wetware.com (drieux, just drieux) writes:>>I remember seeing a little gizmo put on a kidney stone once.  I remember>a little gizmo put on the kidney, and he didn't like it very much.  He>then put the kidney up to worry about it.  It didn't help much either.>>I think it was the putter, not the stone.  He said it made the pain worse.>>There are two ways to deal with this.>>Kidney stones can be made (don't worry, it's not gonna>happen, just hurt your little bladder).  One is


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch 8,400  of  8,481. Loss: 1.2777228355407715.   Elapsed: 1:15:20.
0:  accordingRe: Israeli Terrorism; TLDR: In article <1rd7eo$1a4@usenet.INS.CWRU.Edu> cy779@cleveland.Freenet.Edu (Anas Omran) writes:>	I hope you're not going to argue that the actions taken by the Israeli>police (in Lebanon, and Lebanon) were in the best interests of Israel,>nor are they any worse than the actions of the Israeli government.>The point of all this is that the actions taken by the Israelipolice, (in Lebanon, and now Lebanon) were in the best interests ofIsrael.  Is it not?  In Lebanon you are eitherafraid of going to war (of course), or you are terrified of going to war with theIsraelis.  While it is true that Israeli and Israeli police are sometimes capable of mistakes, that does notmake the war inevitable.  As has been pointed out,

  Average training loss: 0.72
  Training epoch took: 1:16:04

Running Validation...
  Validation Loss: 1.28
  Validation took: 0:02:33

Training complete!
Total traini